In [2]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import spacy

#ipywidgets: Fornisce i widget interattivi (caselle di testo, pulsanti e menù a tendina.)
#IPython.display: Permette di visualizzare e gestire gli output in Jupyter Notebook.
#spacy: Libreria di elaborazione del linguaggio naturale (NLP) utilizzata per analizzare il testo.

#Carica un modello linguistico pre-addestrato di spaCy che verrà usato per analizzare l'input dell'utente.
nlp = spacy.load("en_core_web_sm")

# Funzione che carica le istruzioni di assemblaggio e le suddivide in chunk;
# Legge il file di istruzioni di assemblaggio, lo divide in parti logiche basate su un delimitatore (---) e restituisce una lista di sezioni.
# Se il file non viene trovato, restituisce un messaggio di errore.
def carica_istruzioni(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            contenuto = file.read()
        # Suddividere il testo in chunk basati su fasi logiche dell'assemblaggio
        chunks = contenuto.split('---')
        return chunks
    except FileNotFoundError:
        return ["Errore: Il file non è stato trovato."]

# Funzione che utilizza spaCy per analizzare l'input dell'utente e determinare l'intento cercando parole chiave.
# restitisce un numero che rappresenta la fase dell'assemblaggio.
def analizza_domanda(domanda):
    doc = nlp(domanda.lower())
    if any(token.lemma_ in ["coda", "back"] for token in doc):
        return 1
    elif any(token.lemma_ in ["centro", "center"] for token in doc):
        return 2
    elif any(token.lemma_ in ["tetto", "roof"] for token in doc):
        return 3
    elif any(token.lemma_ in ["base", "bottom"] for token in doc):
        return 4
    elif any(token.lemma_ in ["finale", "complete"] for token in doc):
        return 5
    return 0

# Funzione che restituisce una risposta basata sulla fase corrente dell'assemblaggio. 
# Ogni fase corrisponde a una parte diversa del processo di assemblaggio del prodotto selezionato.
def trova_risposta(fase_corrente, istruzioni):
    prodotto = product_selector.value  # Ottieni il prodotto selezionato
    if fase_corrente == 1:
        return f"Ecco i componenti e i passaggi necessari per assemblare la CODA del prodotto '{prodotto}':\n" + istruzioni[2].strip()
    elif fase_corrente == 2:
        return f"Ecco i componenti e i passaggi necessari per assemblare il CENTRO del prodotto '{prodotto}':\n" + istruzioni[3].strip()
    elif fase_corrente == 3:
        return f"Ecco i componenti e i passaggi necessari per assemblare il TETTO ANTERIORE del prodotto '{prodotto}':\n" + istruzioni[4].strip()
    elif fase_corrente == 4:
        return f"Ecco i componenti e i passaggi necessari per assemblare la BASE ANTERIORE del prodotto '{prodotto}':\n" + istruzioni[5].strip()
    elif fase_corrente == 5:
        return f"Ecco le istruzioni per l'assemblaggio finale del prodotto '{prodotto}':\n" + istruzioni[6].strip() + "\n\nHai completato il processo di assemblaggio! Ora devi fare il controllo qualità."
    else:
        return "Mi dispiace, non ho una risposta per questa domanda. Prova a chiedere informazioni sui moduli di assemblaggio."

# Funzione per gestire l'input testuale dell'utente.
# Analizza la domanda, trova la risposta appropriata e la mostra, ripristinando poi la casella di testo.
def on_submit(change):
    global fase_corrente
    clear_output()  # Pulisci l'output precedente
    
    domanda = input_widget.value
    
    fase_corrente = analizza_domanda(domanda)
    
    risposta = trova_risposta(fase_corrente, istruzioni)
    print(f"AssemblyBot: {risposta}")
    
    # Resetta il campo di input dopo l'invio
    input_widget.value = ''
    display(input_widget, button_indietro, button_avanti, product_selector)
    
    
#Funzioni per navigare nelle fasi di assemblaggio.
#procedi(b): Avanza di una fase nell'assemblaggio.
#indietro(b): Torna indietro di una fase nell'assemblaggio.

def procedi(b):
    global fase_corrente
    clear_output()
    if fase_corrente < 5:
        fase_corrente += 1
    risposta = trova_risposta(fase_corrente, istruzioni)
    print(f"AssemblyBot: {risposta}")
    display(input_widget, button_indietro, button_avanti, product_selector)


def indietro(b):
    global fase_corrente
    clear_output()
    if fase_corrente > 0:
        fase_corrente -= 1
    risposta = trova_risposta(fase_corrente, istruzioni)
    print(f"AssemblyBot: {risposta}")
    display(input_widget, button_indietro, button_avanti, product_selector)

# Funzione per gestire la selezione del file relativa al prodotto scelto dall'utente. 
# Quando l'utente seleziona un prodotto dalla lista, la funzione carica le relative istruzioni e resetta la fase corrente dell'assemblaggio.
def on_product_change(change):
    global fase_corrente, istruzioni
    fase_corrente = 0
    prodotto_selezionato = change['new']
    
    if prodotto_selezionato:
        file_path = file_dict.get(prodotto_selezionato, None)
        if file_path:
            istruzioni = carica_istruzioni(file_path)
            clear_output()
           
            print(f"Il prodotto '{prodotto_selezionato}' ha i seguenti moduli:\n" + 
                  istruzioni[1].strip() + "\nPremi avanti per seguire il manuale passo passo o scrivi cosa vuoi assemblare.")
        else:
            print("Errore: Il file per il prodotto selezionato non è stato trovato.")
    else:
        print("Per favore, seleziona un prodotto dalla lista.")
    
    display(input_widget, button_indietro, button_avanti, product_selector)
#file_dict: Un dizionario che mappa i nomi dei prodotti ai rispettivi file di istruzioni.
#Un widget a tendina che permette all'utente di selezionare il prodotto. 
# L'opzione predefinita è vuota.

#Un dizionario che mappa i nomi dei prodotti ai rispettivi file di istruzioni.
file_dict = {
    "Treno Arancio": "C:\\Users\\marid\\OneDrive\\Desktop\\Progetto SF\\TrenoArancio.txt",
    "Treno Viola": "C:\\Users\\marid\\OneDrive\\Desktop\\Progetto SF\\TrenoViola.txt"
}

#Un widget a tendina che permette all'utente di selezionare il prodotto. 
#L'opzione predefinita è vuota.
product_selector = widgets.Dropdown(
    options=[('', 'Seleziona un prodotto')] + [(k, k) for k in file_dict.keys()],
    description='Seleziona Prodotto:',
    disabled=False,
)

# Inizializza il selettore del prodotto
product_selector.observe(on_product_change, names='value')

# Inizializzazione dei pulsanti e widget
fase_corrente = 0
istruzioni = []  # Inizializzazione per evitare errori al primo avvio

#Widget di Input: Casella di testo dove l'utente inserisce domande o comandi.
#Pulsanti di Navigazione: I pulsanti "Avanti" e "Indietro" permettono all'utente di navigare tra le fasi del processo di assemblaggio.
#Avvio: Viene visualizzato il selettore di prodotto inizialmente, insieme a un messaggio di benvenuto.
input_widget = widgets.Text(description='Modulo:', placeholder='Cerca il modulo da assemblare')
input_widget.on_submit(on_submit)  

button_avanti = widgets.Button(description="Avanti")
button_indietro = widgets.Button(description="Indietro")

button_avanti.on_click(procedi)
button_indietro.on_click(indietro)

# Mostra il selettore di prodotto inizialmente
print("Ciao, sono il tuo assistente AssemblyBot. Seleziona il prodotto che vuoi assemblare.")
display(product_selector)


Ciao, sono il tuo assistente AssemblyBot. Seleziona il prodotto che vuoi assemblare.


C:\Users\marid\AppData\Local\Temp\ipykernel_12360\2314941094.py:149: DeprecationWarning: on_submit is deprecated. Instead, set the .continuous_update attribute to False and observe the value changing with: mywidget.observe(callback, 'value').
  input_widget.on_submit(on_submit)


Dropdown(description='Seleziona Prodotto:', options=(('', 'Seleziona un prodotto'), ('Treno Arancio', 'Treno A…